# Challenge 2 Critique de Films

https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,asc&view=advanced

## Structure de l'URL

### Training

In [1]:
# conda install -c anaconda beautifulsoup4 à faire avant dans la console
import requests
# Librairie BeautifulSoup du package bs4
from bs4 import BeautifulSoup

In [2]:
#Solution Bruno
# Télécharger la page
url = "https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc"
response = requests.get(url)
content = response.content
parser = BeautifulSoup(content, 'html.parser')

print(parser.text[:5])

In [3]:
#Solution Rod
# Télécharger la page
from requests import get
url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&ref_=adv_prv'
response = requests.get(url)
print(response.text[:100])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/


## Structure HTML de la page

In [4]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [5]:
movie_containers = html_soup.find_all('div', class_="lister-item mode-advanced")
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


## Extraire la data pour un seul film

In [6]:
first_movie = movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted">
<span class="certificate">R</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</span>
<span class="genre">
Action, Drama, Sci-Fi            </span>
<

In [7]:
# Titre du film
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>

In [8]:
first_movie.a

<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a>

In [9]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>

In [10]:
first_movie.h3.a

<a href="/title/tt3315342/">Logan</a>

In [11]:
# Titre du film
first_name = first_movie.h3.a.text
first_name

'Logan'

In [12]:
# Année de sortie
first_year = first_movie.h3.find('span', class_="lister-item-year text-muted unbold").text
first_year

'(2017)'

In [13]:
# Note IMDB
first_imdb = float(first_movie.strong.text) #convertir en float (nombre à virgule)
first_imdb

8.1

In [14]:
type(first_imdb)

float

In [15]:
# Note Metacritic
first_metascore = int(first_movie.find('span', class_="metascore favorable").text) #convertir en un nombre entier
first_metascore

77

In [16]:
# Nombre de votes
first_votes = first_movie.find('span', attrs = {'name' : 'nv'})
first_votes

<span data-value="730234" name="nv">730,234</span>

In [17]:
type(first_votes['data-value'])

str

In [18]:
first_votes = int(first_votes['data-value'])
first_votes

730234

## Script pour scraper une seule page

In [19]:
test_movie_metascore = movie_containers[15].find('div', class_='ratings-metascore')
type(test_movie_metascore)

bs4.element.Tag

In [24]:
# On créé des listes vides pour toutes nos informations

names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# On reprend notre data movie_containers pour y extraire l'information
for container in movie_containers:
    
    #Si le film a une note metacritic on l'extrait
    if container.find('div', class_='ratings-metascore') is not None:
        #Le titre du film
        name = container.h3.a.text
        names.append(name) #on ajoute chaque élément name de la boucle à la liste names
        
        #L'année de sortie
        year = container.h3.find('span', class_='lister-item-year text-muted unbold').text
        years.append(year) #on ajoute chaque élément year de la boucle à la liste years
        
        #La note IMDB
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb) #on ajoute chaque élément imdb de la boucle à la liste imdb_ratings
        
        #Metascore
        m_score = container.find('span', class_='metascore').text
        metascores.append(int(m_score)) #on ajoute chaque élément m_score de la boucle à la liste metascores
        
        #Le nombre de votes
        vote = container.find('span', attrs = {'name' : 'nv'})['data-value']
        votes.append(vote) #on ajoute chaque élément vote de la boucle à la liste votes

In [28]:
print(len(names))

41
